In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import VGG
#from torchviz import make_dot
from torchsummary import summary  # 모델 아키텍처 요약을 위한 라이브러리
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

# 장치 설정: GPU 사용 가능 시 GPU로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# 이미지 변환: 센터 크롭 후 125x125로 리사이즈
transform = transforms.Compose([
    transforms.Lambda(lambda img: transforms.functional.center_crop(img, min(img.size))),
    transforms.Resize((125, 125)),
    transforms.ToTensor()
])

# # 이미지 변환: 센터 크롭 후 225x225로 리사이즈
# transform = transforms.Compose([
#     transforms.Lambda(lambda img: transforms.functional.center_crop(img, min(img.size))),
#     transforms.Resize((225, 225)),
#     transforms.ToTensor()
# ])

In [16]:
# 데이터셋 경로 설정
train_data_dir = r'C:\Users\computer\Desktop\Code\KISTI\Flood\dataset\train'
test_data_dir = r'C:\Users\computer\Desktop\Code\KISTI\Flood\dataset\test'
validation_data_dir = r'C:\Users\computer\Desktop\Code\KISTI\Flood\dataset\validation'

# 이미지 파일 유효성 검사 함수 정의 (유효한 확장자 체크)
def is_valid_image_file(file_path):
    return file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.bmp'))

# 훈련, 테스트, 검증 데이터셋 설정, 유효한 이미지 파일만 포함
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform, is_valid_file=is_valid_image_file)
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform, is_valid_file=is_valid_image_file)
val_dataset = datasets.ImageFolder(root=validation_data_dir, transform=transform, is_valid_file=is_valid_image_file)

# 데이터로더 설정: 배치 크기 32, 훈련 데이터는 셔플
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [23]:
class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        self.Conv = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 32, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 64, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )

        self.Fully_Connect = nn.Sequential(
            nn.Linear(128 * 1 * 1, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, num_classes)
        )

        #이미지 크기가 225*225일때
        #     self.Fully_Connect = nn.Sequential(
        #     nn.Linear(128 * 5 * 5, 128),
        #     nn.ReLU(True),
        #     nn.Linear(128, 64),
        #     nn.ReLU(True),
        #     nn.Linear(64, num_classes)
        # )

        
    def forward(self, x):
        x = self.Conv(x)
        x = x.view(x.size(0), -1)
        x = self.Fully_Connect(x)

        return x

In [24]:
model = CustomModel(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [25]:
for epoch in range(100):
    
    running_loss = 0.0
    
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{100}", unit="batch") as pbar:
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            pbar.set_postfix(loss=running_loss / (pbar.n + 1))
            pbar.update(1)
        
        # if i % 100 == 99:
        #     print('Epoch: {}, Iter: {}, Loss: {}'.format(epoch+1, i+1, running_loss/2000))
        #     running_loss = 0.0

Epoch 100/100: 100%|██████████| 33/33 [00:01<00:00, 25.32batch/s, loss=1.61]


In [20]:
PATH = './CustomModel.pth'
torch.save(model.state_dict(), PATH)

In [21]:
model = CustomModel(5)
model.load_state_dict(torch.load(PATH))

C:\Users\computer\AppData\Local\Temp\ipykernel_19448\3817668521.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH))


<All keys matched successfully>

In [22]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(100 * correct / total)

20.0
